<a href="https://colab.research.google.com/github/mudogruer/SLMs/blob/main/phi-2/phi_2_load_adapters_with_hf_peft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U peft bitsandbytes

In [ ]:
from peft import PeftConfig

In [ ]:
peft_model_id = "mudogruer/Phi-2-hf-SciQ-20pc"

In [ ]:
config = PeftConfig.from_pretrained(peft_model_id)

adapter_config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

In [ ]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM

In [ ]:
from huggingface_hub import login
login()

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict = True,
    load_in_4bit = True,
    device_map = "auto"

)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
    padding_size = "right",
    add_eos_token = True,

)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from peft import PeftModel

fine_tuned_model = PeftModel.from_pretrained(
    model,
    peft_model_id
)

adapter_model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline, logging

2024-02-21 20:44:13.501323: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 20:44:13.501453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 20:44:13.661991: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
logging.set_verbosity(logging.CRITICAL)

In [ ]:
from transformers import pipeline
import torch

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    device_map = "auto"

)

In [ ]:
prompt = "<s>Instruct:"+"What is the heaviest element?"+"/Output:"
#prompt = "What is the heaviest element?"

# torch.backends.cuda.enable_mem_efficient_sdp(False)
# torch.backends.cuda.enable_flash_sdp(False)
# torch.cuda.memory_summary(device=None, abbreviated=False)

sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 30,
    temperature = 0.2,
    top_k = 2,
    top_p = 1.0,
)


In [ ]:
print(sequence[0]['generated_text'])

<s>Instruct:What is the heaviest element?/Output:Uranium</s>
    
    
    
    
    
    
    
    
    
    
    
    


In [ ]:
def Sequence(promt):
    sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 30,
    temperature = 0.1,
    top_k = 2,
    top_p = 1.0,
    pad_token_id=tokenizer.eos_token_id
    )
    return sequence

In [ ]:
prompt = "<s>Instruct:"+"What is the heaviest element?"+"/Output:"

print(Sequence(prompt)[0]['generated_text'])

<s>Instruct:What is the heaviest element?/Output:Uranium</s>
    
    
    
    
    
    
    
    
    
    
    
    


In [ ]:
from datasets import load_dataset

dataset = load_dataset("sciq")

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset sciq downloaded and prepared to /root/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})

In [ ]:
dataset_validation = load_dataset('sciq',split='validation[:20%]') #first %20 of validation data

In [ ]:
dataset_validation

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 200
})

In [ ]:
answers = []
for question in dataset_validation["question"]:
    prompt = "<s>Instruct:"+question+"/Output:"
    output_sequences = Sequence(prompt)[0]['generated_text']
    question_and_answer = output_sequences.split("/Output:")
    answer = question_and_answer[1]
    answers.append(answer)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
!pip install -q evaluate bert-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from evaluate import load
import numpy as np
from bert_score import score

bertscore = load("bertscore")
predictions = answers
references = dataset_validation["correct_answer"]
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
print("precision: ",round(np.mean(list(results["precision"])),5))
print("recall: ",round(np.mean(list(results["recall"])),5))
print("f1: ",round(np.mean(list(results["f1"])),5))

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

precision:  0.75496
recall:  0.83209
f1:  0.78678
